<a href="https://colab.research.google.com/github/vtsimoes/class_victor_dataset/blob/main/G_ULMFiT_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalação das bibliotecas

In [ ]:
!pip install wandb

In [ ]:
!pip install fastai==1.0.61

In [ ]:
!pip install spacy==2.3.9

In [ ]:
!pip show spacy

In [ ]:
!pip install colormap

## Carregamento das bibliotecas

In [ ]:
import pickle
from google.colab import drive
import os
import IPython
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from multiprocessing import cpu_count
from fastai.utils.ipython import *
from fastai.basics import *
import re
import warnings
import wandb
from fastai.metrics import *
from fastai.metrics import CMScores
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from datetime import datetime, timedelta
import matplotlib.cm as cm
import time
%matplotlib inline

In [ ]:
import pickle
from google.colab import drive
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef

## Nome do treinamento e locais de salvamento

In [ ]:

path_dados = '' #path com dados pre-processados
path_model = '' #path com modelo ULMFiT com ajuste fino
path_results = path_model + 'Resultados/'
#Montando o google drive para obter os dados
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def save_pickle(data_to_save,path='',filename='file.pkl'):
  open_file = open(path + filename, "wb")
  pickle.dump(data_to_save, open_file)
  open_file.close()

def load_pickle(path='',filename='file.pkl'):
  open_file = open(path + filename, "rb")
  pkl_file = pickle.load(open_file)
  open_file.close()
  return pkl_file

In [ ]:
DATASET = 'O100'
MAX_FEATURES = 70000
DIRECTION = 'BW'
NB_CLASS = 6 if DATASET != 'NO' else 5
CLASSIFICADOR = 'ULMFiT'
EMBEDDING_DIM = MAX_FEATURES  # Size of vocabulary
BATCH_SIZE = 32
EPOCHS = 20
GRAM = 'UNI'
FT = 'VICTOR'
WD = 0.01
LOSSFUNC = 'weighted-xentropy'
MAX_LEN = 3000
NOME_RUNNING = f'{CLASSIFICADOR}-{FT}-{DIRECTION}-{BATCH_SIZE}-{GRAM}-{DATASET}-{MAX_LEN}'
experiment = {'CLASSIFICADOR':CLASSIFICADOR,'FINE_TUNNING':FT,'BATCH_SIZE':BATCH_SIZE,'GRAM':GRAM,'LOSSFUNC':LOSSFUNC,'WD':WD,'EPOCHS':EPOCHS,'HISTORY_FILE':f'{NOME_RUNNING}'}
path_experiment = path_results + NOME_RUNNING
NOME_RUNNING

In [ ]:
if not os.path.exists(path_experiment):
  os.makedirs(path_experiment)

In [ ]:
bs = BATCH_SIZE
data_path = Path(path_model)
print(data_path)

drive/MyDrive/MestradoTucurui/Experiments/ULMFiT


In [ ]:
lang = 'pt'
path = data_path
modelsdir = path/Path('ulmfit_portuguese/')

In [ ]:
torch.cuda.set_device(0)
!nvidia-smi

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import joblib
if DIRECTION == 'FW':
  with open(path / 'ulmfit_portuguese/vocab-fwd.joblib', "rb") as f:
    vocab = joblib.load(f)
else:
  with open(path / 'ulmfit_portuguese/vocab-bwd.joblib', "rb") as f:
    vocab = joblib.load(f)


In [ ]:
import gc
gc.collect()

16

## Carregamento dos Dados

In [ ]:
def get_dataset(DATASET,train,valid,test,train_O10=None,valid_O10=None):
  if DATASET == 'NO':
    train = train.loc[train.document_type != 'outros']
    valid = valid.loc[valid.document_type != 'outros']
    test = test.loc[test.document_type != 'outros']
  elif DATASET == 'O10' and train_O10 is not None and valid_O10 is not None:
    train_filename = train.loc[train.file_name.isin(train_O10)]
    train = pd.concat([train_filename,train.loc[train.document_type != 'outros']], ignore_index=True, sort=False)
    valid_filename = valid.loc[valid.file_name.isin(valid_O10)]
    valid = pd.concat([valid_filename,valid.loc[valid.document_type != 'outros']], ignore_index=True, sort=False)
  return train,valid,test

In [ ]:
train_O10_filenames = load_pickle(path=path_dados,filename='filenames_train_O10.pkl')
valid_O10_filenames = load_pickle(path=path_dados,filename='filenames_valid_O10.pkl')

In [ ]:
doc_train = load_pickle(path = path_dados,filename='train_processed_concat_pages_sw.pkl')
doc_valid = load_pickle(path = path_dados,filename='valid_processed_concat_pages_sw.pkl')
doc_test = load_pickle(path = path_dados,filename='test_processed_concat_pages_sw.pkl')

In [ ]:
doc_train,doc_valid,doc_test = get_dataset(DATASET,doc_train,doc_valid,doc_test,train_O10=None,valid_O10=None)

In [ ]:
doc_train.rename(columns={'body':'text','document_type':'label'},inplace=True)
doc_valid.rename(columns={'body':'text','document_type':'label'},inplace=True)
doc_test.rename(columns={'body':'text','document_type':'label'},inplace=True)

## Montagem dos datasets para treinamento

In [ ]:
doc_train['is_valid'] = False
doc_valid['is_valid'] = True
df_train = doc_train.append(doc_valid,ignore_index=True)
df_train.columns

In [ ]:
%%time
backwards = True if DIRECTION == 'BW' else False
data_clas = (TextList.from_df(df_train, path, vocab=vocab, cols='text').split_from_df(col='is_valid').label_from_df(cols='label').databunch(bs=bs, num_workers=14, backwards=backwards))

In [ ]:
data_clas.path = Path(path_model+'/Dados')

PosixPath('drive/MyDrive/MestradoTucurui/Experiments/ULMFiT/Dados')

In [ ]:
only_new_vocab = list(set(data_clas.vocab.itos) - set(vocab.itos))
only_old_vocab = list(set(vocab.itos) - set(data_clas.vocab.itos))
both_vocab = list(set(vocab.itos) & set(data_clas.vocab.itos))
len(only_new_vocab),len(only_old_vocab),len(both_vocab)

In [ ]:
%%time
data_clas.save(f'{DATASET}_textlist_60k_ulmfit_{DIRECTION}')

In [ ]:
data_clas.show_batch()

## Cáculo de métricas

In [ ]:
@np_func
def f1micro(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='micro')

@np_func
def f1macro(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='macro')

@np_func
def f1weighted(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='weighted')

@np_func
def bacca(inp,targ): return balanced_accuracy_score(targ, np.argmax(inp, axis=-1), adjusted=True) #0 random 1 perfect

@np_func
def bacc(inp,targ): return balanced_accuracy_score(targ, np.argmax(inp, axis=-1), adjusted=False) # 1 perfect

@np_func
def f1binary(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='binary')

@np_func
def mcc(inp,targ): return matthews_corrcoef(targ, np.argmax(inp, axis=-1))

In [ ]:
beep()

## Carregamento do modelo pré-treinado

In [ ]:
from torch.autograd import backward
all_metrics = [accuracy, error_rate, mcc,f1macro,f1weighted,f1micro]
learn_c = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.3,
                                  metrics=all_metrics, lin_ftrs=[100],max_len=MAX_LEN).to_fp16()

In [ ]:
learn_c.path = Path(path_model)
if DIRECTION == 'FW':
  learn_c.load_encoder('encoder_ft_victor_fw')
else:
  learn_c.load_encoder('encoder_ft_victor_bw')
  print('Carregando modelo BW')

In [ ]:
learn_c.loss_func,learn_c.path

In [ ]:
num_trn = len(data_clas.train_ds.x)
num_val = len(data_clas.valid_ds.x)
num_trn, num_val, num_trn+num_val

In [ ]:
trn_LabelCounts = np.unique(data_clas.train_ds.y.items, return_counts=True)[1]
val_LabelCounts = np.unique(data_clas.valid_ds.y.items, return_counts=True)[1]
trn_LabelCounts, val_LabelCounts

In [ ]:
trn_weights = [1 - count/num_trn for count in trn_LabelCounts]
val_weights = [1 - count/num_val for count in val_LabelCounts]
trn_weights, val_weights

In [ ]:
loss_weights = torch.FloatTensor(trn_weights).cuda()
learn_c.loss_func = CrossEntropyFlat(weight=loss_weights )
learn_c.loss_func,learn_c.loss_func.weight

### Identificando o melhor parâmetro de taxa de aprendizado

In [ ]:
with gpu_mem_restore_ctx():
    learn_c.lr_find()
learn_c.recorder.plot(suggestion=True)

In [ ]:
lr = 1e-3
print(lr, bs)
wd = WD
experiment['lr'] = lr
print(wd)

0.001 32
0.01


### Definindo Callbacks

In [ ]:
learn_c.path = Path(path_experiment)
callbacks=[SaveModelCallback(learn_c, monitor='valid_loss', name=NOME_RUNNING, mode='min'),CSVLogger(learn = learn_c, filename=experiment['HISTORY_FILE'],append=True),ShowGraph(learn_c),EarlyStoppingCallback(learn=learn_c,monitor='valid_loss',mode='min',patience=5)]

## Executando o Treinamento

In [ ]:
learn_c.freeze();

In [ ]:
tempo1 = time.time()
with gpu_mem_restore_ctx():
    learn_c.fit_one_cycle(int(EPOCHS*0.2), lr, wd=wd,
                          moms=(0.8,0.7),
                          callbacks=callbacks)

In [ ]:
tempo_2 = time.time()
tempo_total = tempo_2-tempo1

In [ ]:
learn_c.freeze_to(-2)
tempo_1 = time.time()
with gpu_mem_restore_ctx():
    learn_c.fit_one_cycle(int(EPOCHS*0.2), slice(lr/(2.6**4),lr), wd=wd,
                          moms=(0.8,0.7),
                          callbacks=callbacks)

In [ ]:
tempo_2 = time.time()
tempo_total = tempo_total + (tempo_2-tempo_1)

In [ ]:
learn_c.freeze_to(-3)
tempo_1 = time.time()
with gpu_mem_restore_ctx():
    learn_c.fit_one_cycle(int(EPOCHS*0.2), slice(lr/2/(2.6**4),lr/2), wd=wd,
                          moms=(0.8,0.7),
                          callbacks=callbacks)

In [ ]:
tempo_2 = time.time()
tempo_total = tempo_total + (tempo_2-tempo_1)

In [ ]:
import gc
gc.collect()

0

In [ ]:
tempo_1 = time.time()
with gpu_mem_restore_ctx():
    learn_c.unfreeze()
    learn_c.fit_one_cycle(int(EPOCHS*0.4), slice(lr/10/(2.6**4),lr/10), wd=wd,
                          moms=(0.8,0.7),
                          callbacks=callbacks)

In [ ]:
tempo_2 = time.time()
tempo_total = tempo_total + (tempo_2-tempo_1)

In [ ]:
target = learn_c.data.classes
save_pickle(learn_c.data.classes,path=path_experiment,filename='/label_classes.pkl')

In [ ]:
learn_c.model._forward_hooks = OrderedDict()
learn_c.to_fp32().export(NOME_RUNNING+'_exported')

#Cálculo das métricas Para Teste

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
def calculate_metrics(y_true,y_pred):
  f1score = f1_score(y_true, y_pred,average='macro')
  scores = precision_recall_fscore_support(y_true, y_pred,average=None)
  confusion = confusion_matrix(y_true, y_pred)
  report = classification_report(y_true,y_pred,digits=5)
  report_as_dict = classification_report(y_true,y_pred,digits=5,output_dict=True)
  mcc_test = matthews_corrcoef(y_true,y_pred)
  #loss = {'val_loss':history.history['val_loss'],'loss':history.history['loss'],'acc':history.history['accuracy'],'val_acc':history.history['val_accuracy']}
  fig, ax = plt.subplots(figsize=(10, 10))
  ConfusionMatrixDisplay.from_predictions(y_true,y_pred,values_format='.2f',normalize='true',ax=ax,xticks_rotation='vertical')
  return f1score,scores,confusion,report,report_as_dict,mcc_test

In [ ]:
def print_results(report,f1,mcc_test,time_process):
  print(report)
  print("F1-score Médio de Teste : " + str(f1))
  print("MCC de Teste: " + str(mcc_test))
  print("Tempo de processamento total: " + str(time_process) + ' s')

In [ ]:
def saveModelResults(conf,f1,scores,time,classes,report,mcc,experimento,report_as_dicts,path,y_pred,y_true,loss,decimal_preds,tipo="results"):
  score = {'confusion':conf,'f1score':f1,'scores':scores,'mcc_test':mcc,'time':time,'report':report, 'report_as_dict':report_as_dicts,'experimento': experimento,'classes':classes,'y_pred':y_pred,'y_true':y_true,'decimal_preds':decimal_preds}
  filename = tipo+'.pkl'
  save_pickle(score,path=path,filename=filename)
  return score

### Carregando dados de Teste

In [ ]:
target_dict = {'acordao_de_2_instancia':0,
 'agravo_em_recurso_extraordinario':1,
 'despacho_de_admissibilidade':2,
 'outros': 3,
 'peticao_do_RE':4,
 'sentenca':5}

In [ ]:
#df=doc_test
doc_test['document_type_name'] = doc_test['label']
doc_test.replace({"label": target_dict},inplace=True)
doc_test.document_type_name.unique()

array(['outros', 'despacho_de_admissibilidade', 'sentenca', 'agravo_em_recurso_extraordinario', 'peticao_do_RE',
       'acordao_de_2_instancia'], dtype=object)

In [ ]:
learn_c.data.add_test(doc_test.text.values)

### Executando Teste

In [ ]:
y_test = doc_test.document_type_name.values
tempo_1 = time.time()
#preds,y,losses = learn_fwd.get_preds(ds_type=DatasetType.Test,with_loss=True,ordered=True) #preds = valores decimais, y = inteiros indicando a classe, mas vale apenas para treino e validação
preds,y,losses = learn_c.get_preds(ds_type=DatasetType.Test,with_loss=True,ordered=True) #preds = valores decimais, y = inteiros indicando a classe, mas vale apenas para treino e validação
infer = (time.time() - tempo_1)/len(y_test)
infer

In [ ]:
predictions = np.argmax(preds, axis = 1)
y_pred = predictions.numpy()

Convertendo números para labels

In [ ]:
y_pred_label = [target[pred] for pred in y_pred]
f1score,scores,confusion,report,report_as_dict,mcc_test = calculate_metrics(y_test,y_pred_label)

In [ ]:
tempo = {'train':tempo_total,'infer':infer}
dict_results = saveModelResults(confusion,f1score,scores,tempo,target,report,mcc_test,NOME_RUNNING,report_as_dict,path_experiment+'/',y_pred_label,y_test,None,preds,tipo="results")

In [ ]:
print_results(report,f1score,mcc_test,tempo_total)

# Executando Resultados para BW e FW

In [ ]:
NOME_RUNNING = f'{CLASSIFICADOR}-{FT}-FWBW-{BATCH_SIZE}-{GRAM}-{DATASET}-{MAX_LEN}'
NOME_RUNNING

'ULMFiT-VICTOR-FWBW-32-UNI-O100-3000'

In [ ]:
path_experiment = path_results + NOME_RUNNING
if not os.path.exists(path_experiment):
  os.makedirs(path_experiment)
path_experiment

'drive/MyDrive/MestradoTucurui/Experiments/ULMFiT/Resultados/ULMFiT-VICTOR-FWBW-32-UNI-O100-3000'

In [ ]:
results_fw = load_pickle(path=path_results+'ULMFiT-ELIAS-FW-64-UNI-O100/',filename='results.pkl')
results_bw = load_pickle(path=path_results+'ULMFiT-ELIAS-BW-64-UNI-O100/',filename='results.pkl')

In [ ]:
import torch

In [ ]:
predictions_bw_fw = (results_fw['decimal_preds'] + results_bw['decimal_preds'])/2
preds_bw_fw = torch.argmax(predictions_bw_fw,axis=1)
y_pred = preds_bw_fw.numpy()

In [ ]:
target = results_fw['classes']
y_test = results_fw['y_true']

In [ ]:
y_pred_label = [target[pred] for pred in y_pred]
f1score,scores,confusion,report,report_as_dict,mcc_test = calculate_metrics(y_test,y_pred_label)

In [ ]:
print_results(report,f1score,mcc_test,0)

In [ ]:
tempo = {'train':results_fw['time']['train']+results_bw['time']['train'],'infer':results_fw['time']['infer']+results_bw['time']['infer']}
dict_results = saveModelResults(confusion,f1score,scores,tempo,target,report,mcc_test,NOME_RUNNING,report_as_dict,path_experiment+'/',y_pred_label,y_test,None,predictions_bw_fw,tipo="results")